In [12]:
import pandas as pd
import pickle
import urllib

In [19]:
# load vocabularies
bidecades_vocabs_counts_df = pickle.load(urllib.request.urlopen("https://github.com/kasev/mops/raw/master/data/bidecades_vocabs_counts_df.pickle"))
bidecades_vocabs_counts_df.head(5)

,1900-1919,1920-1939,1940-1959,1960-1979,1980-1999,2000-2019
man,11218.0,8342.0,24108.0,39199.0,48435.0,100670.0
new,10183.0,10192.0,23469.0,42092.0,97421.0,293457.0
Jesus,9650.0,4911.0,12134.0,23416.0,49765.0,215687.0
et,9325.0,11818.0,20805.0,35969.0,49326.0,73464.0
God,9248.0,6276.0,26328.0,45751.0,99505.0,293240.0


In [20]:
# explore the counts on your own
bidecades_vocabs_counts_df.loc["man"]

1900-1919     11218.0
1920-1939      8342.0
1940-1959     24108.0
1960-1979     39199.0
1980-1999     48435.0
2000-2019    100670.0
Name: man, dtype: float64

In [15]:
# download the matrices
urllib.request.urlretrieve("https://github.com/kasev/mops/raw/master/data/cooc_norm_dict.pickle", "temp.pickle")
with open("temp.pickle", "rb") as f:
    cooc_norm_dict = pickle.load(f)

In [23]:
cooc_norm_dict.keys()

dict_keys(['1900-1919', '1920-1939', '1940-1959', '1960-1979', '1980-1999', '2000-2019'])

In [25]:
# transform the matrcices into dataframes (much more memory consuming...)
for bidecade, cooc in cooc_norm_dict.items():
    try:
        cooc_norm_df = pd.DataFrame(cooc.todense())
        cooc_norm_df / cooc.max()
        cooc_norm_df.columns = bidecades_vocabs_counts_df.index
        cooc_norm_df.index = bidecades_vocabs_counts_df.index
        cooc_norm_dict[bidecade] = cooc_norm_df
    except:
        pass

In [27]:
cooc_norm_dict["1980-1999"].sort_values("Paul", ascending=False)

,man,new,Jesus,et,God,Paul,make,church,life,see,...,angle,substantive,noteworthy,Kevin,locus,Ian,gesture,metaphorical,layer,less
Paul,37.591206,19.827310,340.144298,67.904433,187.480555,64243.168622,391.246234,144.828710,242.478575,532.952291,...,0.000000,0.000000,1.05304,0.0,0.000000,0.0,0.000000,1.593666,0.0,0.000000
letter,30.454707,17.903023,3.960803,0.000000,6.305044,3384.586931,51.795131,15.934704,97.528689,163.889360,...,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
Apostle,0.000000,0.000000,9.785745,0.000000,7.135993,1630.807205,6.085162,0.000000,6.146624,4.009407,...,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
use,49.490922,37.482732,118.766885,0.000000,114.165990,1355.097080,913.578752,63.491026,14.995467,38.910173,...,0.000000,6.610575,0.00000,0.0,1.738599,0.0,0.832223,35.925463,0.0,7.902183
epistle,0.000000,0.000000,0.000000,0.000000,0.904978,1072.419443,2.835295,0.000000,6.576420,18.826563,...,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
opposite,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.876012,0.000000,0.000000,2.225970,...,0.990307,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
Simeon,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
withdraw,0.000000,0.000000,3.130978,0.000000,3.648156,0.000000,0.000000,0.806888,1.204763,0.000000,...,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
vote,3.056514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.857702,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
